In [4]:
from random import choice

from game import valid, win_condition, next_player, game_updater, get_optimal, Board

liklehood_boards = []

for idx in range(100):
    initial_board, initial_turn, initial_player, initial_winning_condition = (
        game_updater(moves=[])
    )

    liklehood_board: Board = [row[:] for row in initial_board]
    initial_moves = valid(initial_board)

    for initial_move in initial_moves:
        n = 100
        x = 0
        o = 0
        turns = []

        for m in range(n):
            board = [row[:] for row in initial_board]
            turn = initial_turn
            player = initial_player
            winning_condition = initial_winning_condition

            while not winning_condition:
                opponent = "O" if player == "X" else "X"
                valid_moves = valid(board)
                winning_moves = get_optimal(board, valid_moves, player)
                blocking_moves = get_optimal(board, valid_moves, opponent)
                if len(winning_moves):
                    valid_moves = winning_moves
                if len(blocking_moves):
                    valid_moves = blocking_moves
                if initial_move and turn == initial_turn:
                    valid_moves = [initial_move]
                if len(valid_moves):
                    i, j = choice(valid_moves)
                    board[i][j] = player
                    winning_condition = win_condition(board, player)
                    if winning_condition:
                        break
                    else:
                        player = next_player(player)
                        if player == "X":
                            turn += 1
                else:
                    winning_condition = "tie"

            if player == "X":
                x += 1
            if player == "O":
                o += 1
            turns.append(turn)

        u, v = initial_move
        liklehood_board[u][v] = x / (x + o)
    liklehood_boards.append(liklehood_board)

In [43]:
from statistics import median, stdev

liklehood_data = [
    [[], None, None, None, None, None, []],
    [[], None, None, None, None, None, []],
    [[], None, None, None, None, None, []],
    [[], None, None, None, None, None, []],
    [[], None, None, None, None, None, []],
    [[], None, None, None, None, None, []],
    [[], None, None, None, None, None, []],
]

for board in liklehood_boards:
    for i, row in enumerate(board):
        for j, value in enumerate(row):
            if value:
                liklehood_data[i][j].append(value)

liklehood_stats = [
    [(median(item), stdev(item)) if isinstance(item, list) else item for item in row]
    for row in liklehood_data
]

for row in liklehood_stats:
    print(row)

[(0.52, 0.04663754502609037), None, None, None, None, None, (0.53, 0.04685028809889535)]
[(0.52, 0.04555870893249676), None, None, None, None, None, (0.5, 0.05108914767444266)]
[(0.49, 0.04979260015607772), None, None, None, None, None, (0.49, 0.04582917336493352)]
[(0.45, 0.04787135538781691), None, None, None, None, None, (0.44, 0.049612131947373884)]
[(0.49, 0.05324955541143186), None, None, None, None, None, (0.5, 0.053421423463330564)]
[(0.51, 0.04792871390055878), None, None, None, None, None, (0.51, 0.05229713088678861)]
[(0.53, 0.04923905816001001), None, None, None, None, None, (0.52, 0.05050012501234652)]
